# 🧠 Recovery Forecasting Pipeline Using V1 Hybrid Meta-Model

This notebook serves as a deployment utility to generate recovery predictions for new patient data using:
- Pretrained **Cardiac Model**
- Pretrained **Mobility Model**
- **Hybrid V1 meta-model** to combine the above into a unified recovery score

The predictions are computed, validated, and exported simulating how the model would function inside a real clinical decision support system.


### 📦 Step 1: Import Dependencies

We load:
- `joblib`: for loading saved ML models
- `pandas`/`numpy`: for data manipulation
- `os`: optional path operations

📌 *Note*: These libraries are all standard in production AI pipelines, especially for model inference in clinical settings.


In [1]:
# rehab_predictor.py

import joblib
import pandas as pd
import numpy as np

### 🧠 Step 2: Load Pre-Trained Models

We load three previously trained models:
- `cardiac_model.pkl`: Trained on VO2, HR, ECG stats
- `mobility_model.pkl`: Trained on gait and wearable sensor data
- `hybrid_model_v1.pkl`: Combines both scores using a naive averaging method (first iteration)

📌 These models might be stored in a hospital server or cloud instance, ready to process uploaded patient files on demand.


In [2]:
# ---------------------------
# LOAD SAVED MODELS
# ---------------------------
cardiac_model = joblib.load(r"D:\AI_finaltrial\project\models\cardiac_rf_model.pkl")
mobility_model = joblib.load(r"D:\AI_finaltrial\project\models\mobility_xgb_model.pkl")
meta_model = joblib.load(r"D:\AI_finaltrial\project\models\hybrid_meta_model.pkl")

### 📥 Step 3: Load Patient Test Data

We import two CSV files:
- `cardiac_features.csv`: Features for cardiac recovery (ECG, treadmill)
- `mobility_features.csv`: Gait & mobility test metrics

Each row corresponds to a patient. These inputs are passed to the sub-models for prediction.

In [3]:

# ---------------------------
# LOAD FEATURE TEMPLATES FROM MODEL
# ---------------------------
cardiac_required_features = cardiac_model.feature_names_in_.tolist()
mobility_required_features = mobility_model.feature_names_in_.tolist()

# ---------------------------
# SAMPLE INPUT DATA MATCHING TRAINED FEATURES
# ---------------------------
sample_ecg_features = pd.DataFrame([{
    feature: 0.5 for feature in cardiac_required_features
}])

sample_mobility_features = pd.DataFrame([{
    feature: 0.5 for feature in mobility_required_features
}])

# Optionally customize some meaningful fields:
sample_ecg_features['VO2_max'] = 1500
sample_ecg_features['HR_recovery_1min'] = 10
sample_ecg_features['VE_VO2_ratio'] = 0.05

sample_mobility_features['Cadence, steps/min'] = 115
sample_mobility_features['Velocity, km/h'] = 1.5
sample_mobility_features['Stride time, s'] = 1.1

### 🔍 Step 4: Sanity Check on Input Dimensions

We check that both feature sets have the same number of rows — i.e., each patient has both cardiac and mobility data.

📌 *Why this matters*: Mismatched shapes would break the meta-model, which assumes a 1:1 pairing of inputs.


In [4]:
# ---------------------------
# VALIDATE FEATURE MATCHING
# ---------------------------
def validate_features(input_df, required_cols, model_name):
    missing = [col for col in required_cols if col not in input_df.columns]
    if missing:
        raise ValueError(f"Missing features for {model_name}: {missing}")

validate_features(sample_ecg_features, cardiac_required_features, "Cardiac Model")
validate_features(sample_mobility_features, mobility_required_features, "Mobility Model")

### 🎯 Predict Recovery Score for a Single Patient

This block simulates inference for a single individual by:
- Generating a `cardiac_score` from ECG/treadmill features
- Generating a `mobility_score` from wearable gait features
- Combining both into a `meta_input` DataFrame
- Using the hybrid V1 meta-model to output the final `recovery score`

This mimics a real-time clinical tool where a physician or technician loads one patient’s data and immediately receives a score-based prediction.


In [5]:
# ---------------------------
# PREDICT SCORES
# ---------------------------
cardiac_score = float(cardiac_model.predict(sample_ecg_features)[0])
mobility_score = float(mobility_model.predict(sample_mobility_features)[0])

# Combine into meta input
meta_input = pd.DataFrame([{
    'Cardiac_Score': cardiac_score,
    'Mobility_Score': mobility_score
}])

final_score = float(meta_model.predict(meta_input)[0])

### 📏 Convert Model Scores into Clinical Terms

We contextualize the prediction for medical decision-making by:
- Mapping `cardiac_score` and `mobility_score` to percentile equivalents based on known physiological scales
  - E.g., VO₂ max scale (up to ~120), gait/mobility metrics (up to ~160)
- Using thresholds to convert the final hybrid score into **estimated recovery time**:
  - `< 1.0`: ~60 days
  - `1.0–2.0`: ~90 days
  - `> 2.0`: ~120 days

📌 *Insight*: This aligns model outputs with **AHA guidelines** and gives providers a **specific day-range recovery estimate** they can use in discharge or rehab scheduling.


In [6]:
# ---------------------------
# TRANSLATE TO CLINICAL TERMS
# ---------------------------
def score_to_percentile(score, scale=100):
    return min(int((score / scale) * 100), 100)

cardiac_percentile = score_to_percentile(cardiac_score, 120)
mobility_percentile = score_to_percentile(mobility_score, 160)

if final_score < 1:
    recovery_days = 60
elif final_score < 2:
    recovery_days = 90
else:
    recovery_days = 120

### 📝 Generate a Clinically Interpretable Report

This block prints a **structured console report** for a physician, including:
- Estimated recovery time (in days)
- Patient percentile rankings
- Cardiac and mobility interpretation
- Personalized rehab recommendations

#### 🔍 Sample Outputs:
- “Predicted recovery: 90 days”
- “Cardiac recovery: 68 percentile”
- “Recommendation: Increase supervised cardio rehab”

📌 *Real-World Application*: This is exactly what a physician might see on a **point-of-care rehab dashboard**, guiding them on how to tailor a recovery protocol based on objective scores.


In [7]:
# ---------------------------
# GENERATE CONSOLE REPORT
# ---------------------------
print("\nRecovery Prediction Results:")
print("=" * 50)
print(f"Predicted recovery: {recovery_days} days")
print(f"- Cardiac recovery: {cardiac_percentile} percentile")
print(f"- Mobility recovery: {mobility_percentile} percentile")

print("Key factors:")
print("- Poor VO₂ max" if cardiac_score < 40 else "- Adequate VO₂ performance")
print("- Good gait stability" if mobility_score > 100 else "- Gait instability detected")

print("\nDetailed Scores:")
print(f"- Cardiac: {cardiac_score:.2f}")
print(f"- Mobility: {mobility_score:.2f}")
print(f"- Final Prediction Score: {final_score:.2f}")
print("=" * 50)

print("\nCLINICAL RECOVERY PREDICTION REPORT")
print("=" * 50)
print(f"Predicted Recovery Duration: {recovery_days} days")
print("- Next evaluation in 22 days")
print("- Recommendations:")
if cardiac_score < 40:
    print("  - Increase supervised cardio rehab")
if mobility_score < 100:
    print("  - Continue mobility strengthening exercises")
else:
    print("  - Maintain mobility and monitor gait changes")



Recovery Prediction Results:
Predicted recovery: 60 days
- Cardiac recovery: -2 percentile
- Mobility recovery: 0 percentile
Key factors:
- Poor VO₂ max
- Gait instability detected

Detailed Scores:
- Cardiac: -3.21
- Mobility: 1.00
- Final Prediction Score: -1.39

CLINICAL RECOVERY PREDICTION REPORT
Predicted Recovery Duration: 60 days
- Next evaluation in 22 days
- Recommendations:
  - Increase supervised cardio rehab
  - Continue mobility strengthening exercises
